# Implementation of the algorithm used in PianoText
To get a mapping between a piano and common letters we have to:
- read in midi files of melodies
- build uni-grams of the appearance of notes
- build bi-grams of the transition of notes
- read in a text corpus
- build uni-grams of the letters
- build bi-grams of the letters
- apply the algorithm described in PianoText

The link to the text dataset in the PianoText paper is dead so we used the 'AUTHORS'-Folder from the 'Classic Literature in ASCII' dataset by Myles O'Neill (https://www.kaggle.com/datasets/mylesoneill/classic-literature-in-ascii?resource=download). The piano dataset used in the paper is not publicly available, other public piano datasets are not generated by side-reading (as in the paper) and consist of many notes played at the same time, so they are not suitable for generating bi-grams of transitions. We used the 'Melody' lines from the 'POP 909' dataset (from Wang, Z. et al. (2020). Pop909: A pop-song dataset for music arrangement generation. arXiv preprint arXiv:2008.07142. | https://github.com/music-x-lab/POP909-Dataset) as a proxy dataset for the proof of concept of the algorithm. These melody lines are good as a proxy dataset because there is only one note at a time like in sight reading. However, these melodies are intended to be performed by a singer, so the mapping is not the same as for a piano. So, we used this notebook as a proof of concept for the implementation of the algorithm but used the mapping presented in the paper for the actual implementation.

In [1]:
import mido
import os
from nltk.probability import FreqDist

lower_case_letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# read in midi files

In [2]:
melody_data = []
for root, _, files in os.walk('POP909'):
    for file in files:
        name = file.split('.')[0]
        if(len(name) == 3):
            mid = mido.MidiFile(f'POP909/{name}/{file}')
            for track in mid.tracks:
                melody = []

                if(track.name == 'MELODY'):
                    for msg in track:
                        if hasattr(msg, 'velocity'):
                            if(msg.velocity > 0):
                                melody.append(msg)
                    melody_data.append(melody)

# create uni-grams for notes

In [3]:
uni_grams = {}
for melody in melody_data:
    for note in melody:
        # we only need the notes played
        if(note.type == 'note_on'):
            if f'{note.note}' in uni_grams:
                uni_grams[f'{note.note}'] = uni_grams[f'{note.note}'] + 1
            else:
                uni_grams[f'{note.note}'] = 1
number_of_notes = 0
#generate probabilities
for apperences in uni_grams.values():
    number_of_notes += apperences
for key in uni_grams.keys():
    uni_grams[key] = uni_grams[key] / number_of_notes
# sort the uni-grams
# source for sorting the code: https://www.freecodecamp.org/news/sort-dictionary-by-value-in-python/
uni_grams = sorted(uni_grams.items(), key=lambda x:x[1], reverse=True)
print(uni_grams)

[('71', 0.06288478878428559), ('74', 0.062238424422231056), ('72', 0.06017652210727709), ('69', 0.05911002090988711), ('73', 0.0562143085678828), ('75', 0.053606228366992754), ('76', 0.053286278007775766), ('67', 0.05283382295433759), ('70', 0.049366078151915016), ('77', 0.04689373446705643), ('68', 0.04103767334684235), ('78', 0.03901778471542193), ('79', 0.036930027825985784), ('66', 0.036855695924349514), ('65', 0.03396321540415548), ('64', 0.03266402303642586), ('80', 0.026694848152852245), ('62', 0.025728533431580717), ('63', 0.023799135810847933), ('81', 0.022406220610620413), ('60', 0.016718214224540517), ('82', 0.016527536737734428), ('61', 0.013347424076426122), ('83', 0.012044999886886237), ('84', 0.010888007678808622), ('59', 0.010858921282516167), ('58', 0.007759604166464678), ('57', 0.006780362157952059), ('86', 0.005115973925661634), ('85', 0.004705532555757006), ('55', 0.0038394043106039304), ('56', 0.003047607967087127), ('88', 0.002902175985624857), ('87', 0.0022881298

# create bi grams for notes

In [4]:
bi_grams = {}
notes_of_melody = []
for melody in melody_data:
    melody_notes = []
    for note in melody:
        # we only need the notes played
        if(note.type == 'note_on'):
            melody_notes.append(note.note)
    notes_of_melody.append(melody_notes)
for melody in notes_of_melody:
    # create note transitions
    for i in range(len(melody)):
        if(i + 1 < len(melody) - 1):
            bi_gram = [melody[i], melody[i + 1]]
            if f'{bi_gram}' in bi_grams:
                bi_grams[f'{bi_gram}'] = bi_grams[f'{bi_gram}'] + 1
            else:
                bi_grams[f'{bi_gram}'] = 1
# #generate probabilities
number_of_notes = 0
for apperences in bi_grams.values():
    number_of_notes += apperences
for key in bi_grams.keys():
    bi_grams[key] = bi_grams[key] / number_of_notes
# sort the bi-grams
# source: https://www.freecodecamp.org/news/sort-dictionary-by-value-in-python/
bi_grams = sorted(bi_grams.items(), key=lambda x:x[1], reverse=True)
bi_grams_transitions = []
# remove transitions form one note to its self
for bi_gram in bi_grams:
    note_one_two = bi_gram[0].split(',')
    note_one = note_one_two[0]
    note_two = note_one_two[1]
    note_one = note_one.replace('[', '')
    note_two = note_two.replace(']', '')
    note_two = note_two.replace(' ', '')
    note_one = int(note_one)
    note_two = int(note_two)
    if(note_one != note_two):
        bi_grams_transitions.append(bi_gram)
print(bi_grams_transitions[: 50])

[('[71, 69]', 0.013058955478617058), ('[74, 72]', 0.012834641829619154), ('[76, 74]', 0.012802132605126705), ('[69, 67]', 0.011882121551990378), ('[69, 71]', 0.011374977649908162), ('[75, 73]', 0.011267697209083077), ('[72, 74]', 0.01092309942946311), ('[73, 71]', 0.010812568066188782), ('[67, 69]', 0.010789811609044067), ('[74, 76]', 0.010376944457989954), ('[71, 73]', 0.010097365127354888), ('[73, 75]', 0.010035597600819233), ('[72, 70]', 0.009834040408966044), ('[77, 75]', 0.009567464768127957), ('[70, 72]', 0.00917085222932007), ('[78, 76]', 0.008715723086425773), ('[67, 65]', 0.008702719396628793), ('[75, 77]', 0.008673461094585588), ('[70, 68]', 0.00854992604151428), ('[79, 77]', 0.008036280294533574), ('[65, 67]', 0.007971261845548675), ('[68, 70]', 0.007854228637375855), ('[76, 78]', 0.00727556444141025), ('[68, 66]', 0.0071617821556866764), ('[77, 79]', 0.006492092131142211), ('[66, 64]', 0.006466084751548252), ('[66, 68]', 0.006436826449505047), ('[80, 78]', 0.005822402106597

# Read in text

In [5]:
text_files_safe = []
for root, dirs, files in os.walk('AUTHORS'):
    for dir in dirs:
        for root, _, text_files in os.walk(f'AUTHORS/{dir}'):
            for text_file in text_files:
                if(text_file[-4:] == '.txt'):
                    with open(f'AUTHORS/{dir}/{text_file}') as f:
                        content = f.readlines()
                        text_files_safe.append(content)

# Create text uni grams

In [7]:
letters = []
for t in text_files_safe:
    for line in t:
        for char in line:
            # we only are intrested in the apperence of lower case letters
            if(char in lower_case_letters):
                letters.append(char)
# generate letter frequencys
uni_gram_distribution = FreqDist(letters)
letter_counts = []
for k, v in uni_gram_distribution.items():
    letter_counts.append([k, v])
# calculate the probabilities from the frequencys
apperences = 0
for letter in letter_counts:
    apperences += letter[1]
for letter in letter_counts:
    letter[1] = letter[1] / apperences
letter_counts = sorted(letter_counts, key=lambda x:x[1], reverse=True)
print(letter_counts)

[['e', 0.12730031130753489], ['t', 0.09313182470104242], ['a', 0.07975326393940883], ['o', 0.0781903267933764], ['n', 0.07103014646638407], ['i', 0.06713406315262423], ['s', 0.06340980528350634], ['h', 0.06288542979899193], ['r', 0.059192868739713286], ['d', 0.042241907443057496], ['l', 0.03976524597860653], ['u', 0.029218271782903184], ['c', 0.025396137918185492], ['m', 0.024858787560078248], ['f', 0.0233441433333846], ['w', 0.02187000710187403], ['y', 0.019239978233293417], ['g', 0.019238600111126495], ['p', 0.01725376699065195], ['b', 0.014793012574617069], ['v', 0.009483342439446594], ['k', 0.006911458597610861], ['x', 0.0016158775624660628], ['j', 0.0011128776324130932], ['q', 0.0010200156557610338], ['z', 0.0006085289019414326]]


# Create text bi grams

In [8]:
letter_bi_grams = []
for t in text_files_safe:
    for line in t:
        # get word 'transitions'
        for i in range(len(line)):
            if(i + 1 < len(line) - 1):
                if(line[i] in lower_case_letters and line[i + 1] in lower_case_letters):
                    letter_bi_grams.append(f'{line[i]} - {line[i + 1]}')
# generate 'transition' frequencys
bi_gram_distribution = FreqDist(letter_bi_grams)
letter_counts_bi_grams = []
for k, v in bi_gram_distribution.items():
    letter_counts_bi_grams.append([k, v])
# calculate the probabilities from the frequencys
apperences = 0
for letter in letter_counts_bi_grams:
    apperences += letter[1]
for letter in letter_counts_bi_grams:
    letter[1] = letter[1] / apperences
letter_counts_bi_grams = sorted(letter_counts_bi_grams, key=lambda x:x[1], reverse=True)
print(letter_counts_bi_grams[:30])

[['t - h', 0.04021206397391681], ['h - e', 0.03700136501874384], ['i - n', 0.023551434017478914], ['a - n', 0.02183930283762451], ['e - r', 0.021357721003514074], ['r - e', 0.018413084698847414], ['n - d', 0.01700041780809704], ['o - n', 0.015126513020407404], ['a - t', 0.014308748103900955], ['e - n', 0.013785979863528804], ['o - u', 0.01353493069351734], ['h - a', 0.013506130888021486], ['e - d', 0.012238805226071762], ['i - t', 0.0121202505007849], ['i - s', 0.012094365189586896], ['o - r', 0.011835588774825212], ['h - i', 0.011530468065866804], ['e - s', 0.011426217371804914], ['o - f', 0.011317556587687067], ['t - o', 0.011274471925270832], ['n - g', 0.010955890854489463], ['a - r', 0.01035479557986261], ['a - s', 0.009982699024967437], ['t - e', 0.009839198529407536], ['s - t', 0.009786104879994739], ['s - e', 0.009444783083968306], ['v - e', 0.009118551465655081], ['l - e', 0.008880349079719664], ['m - e', 0.00869017832678499], ['t - i', 0.00848790858765332]]


# Algorithm PianoText
## First loop (generate the first mapping of letters and notes by there frequency)

This part of the algorithm assigns the n-frequent-letter to the n-frequent-note. For every assignment it then checks for all already mapped letters if there a common bi-grams. If there is a common bi-gram the algorithm searches if the mapping of these two letters is a common transition in music, if it is not a frequent transition it reassigns the letter to the next common note an checks again.

In [9]:
mapping = {}

assig_notes_list = uni_grams.copy()
for new_letter in letter_counts:
    for note in assig_notes_list:
        reassignment = False
        assigned_frequency = note[0]
        for letter in mapping.keys():
            #check if word combinations are common
            combination_one = f'{new_letter[0]} - {letter}'
            combination_two = f'{letter} - {new_letter[0]}'
            for bi_gram_letters in letter_counts_bi_grams:
                if(bi_gram_letters[1] < 0.001):
                    break
                if(combination_one == bi_gram_letters[0]):
                    # check if note transition is common
                    bi_gram_current_notes = f'[{assigned_frequency}, {mapping[combination_one[-1]]}]'
                    reassignment = True
                    for bi_gram_notes in bi_grams_transitions:
                        if(bi_gram_notes[0] == bi_gram_current_notes):
                            reassignment = False   
                elif(combination_two == bi_gram_letters[0]):
                    bi_gram_current_notes = f'[{mapping[combination_one[-1]]}, {assigned_frequency}]'
                    reassignment = True
                    for bi_gram_notes in bi_grams_transitions:
                        if(bi_gram_notes[0] == bi_gram_current_notes):
                            reassignment = False
        if not reassignment:
            mapping[new_letter[0]] = assigned_frequency
            assig_notes_list.remove(note)
            break
# source https://www.geeksforgeeks.org/python-convert-dictionary-to-list-of-tuples/
mapping_list = [(k, v) for k, v in mapping.items()]
print(mapping_list)

[('e', '71'), ('t', '74'), ('a', '72'), ('o', '69'), ('n', '73'), ('i', '75'), ('s', '76'), ('h', '67'), ('r', '70'), ('d', '77'), ('l', '68'), ('u', '78'), ('c', '79'), ('m', '66'), ('f', '64'), ('w', '65'), ('y', '80'), ('g', '62'), ('p', '63'), ('b', '81'), ('v', '60'), ('k', '82'), ('x', '61'), ('j', '83'), ('q', '86'), ('z', '84')]


## Second loop (check for common bi-grams)
This part of the algorithm goes to all frequent bi-grams and checks the distance of the mapping. If there is a distance greater than 7 semi-tones it assigns the more frequent letter to another note. It starts the search on the opposite side of the infrequent letter to increase the distance to the already existing letter, then it alternates to find the smallest distance to the infrequent letter. 

In [10]:
taken_positions = []
for element in mapping_list:
    taken_positions.append(element[1])
for bi_gram in letter_counts_bi_grams:
    if(bi_gram[1] < 0.01):
        break
    # get int values of notes
    letter_one_two = bi_gram[0].split(' - ')
    letter_one = letter_one_two[0]
    letter_two = letter_one_two[1]
    if(letter_one == letter_two):
        continue
    # letters now can appeare more then once
    apperences_letter_one = []
    apperences_letter_two = []
    for elements in mapping_list:
        if(elements[0] == letter_one):
            apperences_letter_one.append(elements[1])
        elif(elements[0] == letter_two):
            apperences_letter_two.append(elements[1])
    # find notes with shortest distance
    distance = 10000000
    note_one = None
    note_two = None
    for apperence_one in apperences_letter_one:
        for apperence_two in apperences_letter_two:
            if(int(apperence_one) - int(apperence_two) < distance):
                distance = int(apperence_one) - int(apperence_two)
                note_one = int(apperence_one)
                note_two = int(apperence_two)
    #check distance
    if(distance < -7 or 7 < distance):
        # remap the more frequent letter
        # find the more frequent letter
        remap_letter_one = True
        for letter in letter_counts:
            if(letter == letter_one):
                # first letter is more frequent
                break
            elif(letter == letter_two):
                # second letter is more frequent
                remap_letter_one = False
                break
        # remape letter one
        if(remap_letter_one):
            # if distance is positive note one is higer
            # if distance is negative note two is higer
            # at first we search on the opposite side to increase the distance to the existing note
            # then we search alternating to minimize the distance
            starts_positive = True
            if(distance < 0):
                distance_counter = 1
            else:
                distance_counter = -1
                start_positive = False
            while(True):
                new_position = note_two + distance_counter
                if(f'{new_position}' in taken_positions):
                    if(distance_counter < 0):
                        if(starts_positive):
                            distance_counter -= 1
                            distance_counter *= -1
                        else:
                            distance_counter *= -1
                    else:
                        if(not starts_positive):
                            distance_counter += 1
                            distance_counter *= -1
                        else:
                            distance_counter *= -1
                    continue
                mapping_list.append((letter_one, new_position))
                mapping[letter_one] = f'{new_position}'
                taken_positions.append(f'{new_position}')
                break
        # remap letter two
        elif(not remap_letter_one):
            # if distance is positive note one is higer
            # if distance is negative note two is higer
            # at first we search on the opposite side to increase the distance to the existing note
            # then we search alternating to minimize the distance
            starts_positive = True
            if(distance < 0):
                distance_counter = -1
                starts_positive = False
            else:
                distance_counter = 1
            while(True):
                new_position = note_one + distance_counter
                if(f'{new_position}' in taken_positions):
                    if(distance_counter < 0):
                        if(starts_positive):
                            distance_counter -= 1
                            distance_counter *= -1
                        else:
                            distance_counter *= -1
                    else:
                        if(not starts_positive):
                            distance_counter -= 1
                            distance_counter *= -1
                        else:
                            distance_counter *= -1
                    continue
                mapping_list.append((letter_two, new_position))
                taken_positions.append(f'{new_position}')
                break
print(mapping_list)

[('e', '71'), ('t', '74'), ('a', '72'), ('o', '69'), ('n', '73'), ('i', '75'), ('s', '76'), ('h', '67'), ('r', '70'), ('d', '77'), ('l', '68'), ('u', '78'), ('c', '79'), ('m', '66'), ('f', '64'), ('w', '65'), ('y', '80'), ('g', '62'), ('p', '63'), ('b', '81'), ('v', '60'), ('k', '82'), ('x', '61'), ('j', '83'), ('q', '86'), ('z', '84'), ('o', 85), ('h', 87), ('t', 88), ('n', 59)]
